In [1]:
import csv
import numpy as np
import random
import time
from PIL import Image
from imageio import imwrite,imread
import os
from pylablib.devices import Thorlabs

def mkdir(path):
	folder = os.path.exists(path)
	if not folder:                   
		os.makedirs(path)    

from pylablib.devices import Thorlabs
Thorlabs.list_cameras_tlcam()
cam = Thorlabs.ThorlabsTLCamera(serial="14308")
# cam.close()
cam.set_exposure(0.01) # set exposure to 100ms
# frame = cam.snap() # grab a single frame
cam.set_roi(0,1440,0,1080) # set ROI in the upper left corner
#images = cam.grab(10) # grab 10 frames

i=0
while i<= 0:
    #A = np.array([[0],[0],[0],[0],[0]], dtype=np.int32)
    A = np.random.randint(low=0,high=2000,size=(5,1), dtype=np.int32)   # range form 0 to 3000
    with open('PWM_test.csv', 'w', newline='') as file:
        mywriter = csv.writer(file, delimiter=',')
        mywriter.writerows(A)
    time.sleep(0.1) 
    fig_name = 'pwm='+str(int(A[0]))+'-'+str(int(A[1]))+'-'+str(int(A[2]))+'-'+str(int(A[3]))+'-'+str(int(A[4])) +'-'+str(int(i))
    image_array = cam.grab(1)    # grab 1 frame
    image_array=np.array(image_array)
    image_array=image_array.squeeze()
    image = Image.fromarray(image_array)
    mkdir(r'C:\Users\plasma\Desktop\m1')
    imwrite(r'C:\Users\plasma\Desktop\m1\{fig_name}.png'.format(fig_name=fig_name),image)
    i=i+1
#can use this to add aberration or add it on pi,see pi_control.py file

In [2]:
from skopt import gp_minimize
import pygad
import laserbeamsize as lbs

def get_quality_number(image_array):
    x, y, dx, dy, phi = lbs.beam_size(image_array)
    Ellipticity = min(dx, dy) / max(dx, dy)
    threshold_up_area = (image_array >= np.max(image_array) / 10) + 0  # define a threshold area
    total_pixel = image_array.shape[0] * image_array.shape[1]
    dispersion = total_pixel / np.sum(threshold_up_area)
    quality_number = dispersion * Ellipticity
    return quality_number

def objective(solution):
    A = np.array([[solution[0]], [solution[1]], [solution[2]], [solution[3]], [solution[4]]])
    with open('PWM_test.csv', 'w', newline='') as file:
        mywriter = csv.writer(file, delimiter=',')
        mywriter.writerows(A)
    time.sleep(0.1)
    cam.set_roi(0, 1440, 0, 1080)  # set ROI
    images = cam.grab(1)  # grab 1 frame
    quality_number = get_quality_number(image_array)
    score = -quality_number
    return score

bounds = [(0, 4095)] * 5  # Example bounds for 5 PWM values

# Modify the following parameters as per your requirements
n_calls = 50  # Number of iterations
n_points = 100  # Number of points used for training the surrogate model

res = gp_minimize(objective, bounds, n_calls=n_calls, n_points=n_points)

optimal_pwm_values = res.x

# Use the optimal_pwm_values for controlling the deformable mirror


In [3]:
optimal_pwm_values

[2917, 636, 3862, 1824, 848]

In [4]:
## close camera and are ready to run again
cam.close()

In [5]:
res

          fun: -1.537144369040655
            x: [2917, 636, 3862, 1824, 848]
    func_vals: [-1.537e+00 -1.537e+00 ... -1.537e+00 -1.537e+00]
      x_iters: [[2917, 636, 3862, 1824, 848], [2009, 2286, 188, 1789, 3207], [2961, 3677, 421, 3214, 1292], [1690, 1505, 174, 1492, 540], [3700, 3586, 757, 3835, 1746], [476, 1049, 1831, 1044, 3973], [489, 1847, 1505, 355, 950], [2854, 150, 2033, 3594, 2897], [870, 1446, 4087, 1944, 83], [2229, 1735, 3256, 2241, 1486], [0, 4095, 4095, 4095, 4095], [4095, 4095, 4095, 0, 4095], [2162, 1238, 894, 1605, 577], [2019, 2362, 1174, 845, 3982], [205, 2121, 2220, 2534, 1870], [3802, 2567, 2432, 877, 3868], [319, 1187, 3830, 20, 412], [24, 427, 581, 89, 3644], [2967, 1815, 3865, 3538, 1612], [1198, 603, 233, 3111, 2517], [2340, 2333, 1800, 2778, 2295], [3957, 3295, 293, 3764, 3619], [622, 3163, 860, 836, 3429], [1888, 1394, 4044, 3455, 212], [1690, 3317, 3001, 3666, 753], [671, 2576, 3646, 2943, 2565], [3318, 662, 3909, 572, 2469], [3980, 3376, 1012, 2407,